In [1]:
#execution requirements.txt
#! pip install -r requirements.txt

# Librairie pddl

### Exploration librairie

In [2]:
from pddl.parser.domain import DomainParser
from pddl.parser.problem import ProblemParser
import pddl
from itertools import product
import copy

#import data/domain.pddl in str

domain_str = open("data/domain.pddl").read()
problem_str = open("data/problem.pddl").read()  

# print(domain_str)
# print(problem_str)


domain = DomainParser()(domain_str)
problem = ProblemParser()(problem_str)

init_state = copy.deepcopy(list(problem.init))
goal_state = copy.deepcopy(list(problem.goal.__getstate__()['_operands']))

# PROBLEM
print("Initial : ", init_state)
print("Goal : ", goal_state)
print("0bjects :", list(problem.objects))


# DOMAIN
print("Predicates : ", list(domain.predicates))
print("Actions : ", list(domain.actions))
print("Action 0 Name : ",list((domain.actions))[0].name)
print("Action 0 Precondition : ",list(list((domain.actions))[0].precondition.operands))
print("Action 0 Effect : ",list(list((domain.actions))[0].effect.operands))
print("Action 0 Parameters : ",list(list((domain.actions))[0].parameters))


Initial :  [Predicate(smaller, d2, p2), Predicate(smaller, d2, p1), Predicate(smaller, d0, p0), Predicate(on, d2, d3), Predicate(on, d0, d1), Predicate(smaller, d1, p0), Predicate(clear, p2), Predicate(smaller, d1, d3), Predicate(smaller, d0, d3), Predicate(smaller, d3, p0), Predicate(smaller, d1, p1), Predicate(smaller, d0, p1), Predicate(smaller, d3, p1), Predicate(on, d1, d2), Predicate(smaller, d0, p2), Predicate(smaller, d1, p2), Predicate(smaller, d2, d3), Predicate(smaller, d0, d1), Predicate(smaller, d3, p2), Predicate(on, d3, p0), Predicate(clear, p1), Predicate(smaller, d0, d2), Predicate(smaller, d2, p0), Predicate(clear, d0), Predicate(smaller, d1, d2)]
Goal :  [Predicate(on, d0, d1), Predicate(on, d1, d2), Predicate(on, d2, d3), Predicate(on, d3, p2), Predicate(clear, p0), Predicate(clear, p1), Predicate(clear, d0), Predicate(smaller, d0, d1), Predicate(smaller, d0, d2), Predicate(smaller, d0, d3), Predicate(smaller, d0, p0), Predicate(smaller, d0, p1), Predicate(smaller, 

### Fonction pour générer toutes les actions possibles du problème

In [3]:
def generate_klist(variables, k):
    return list(product(variables, repeat=k))


def all_actions(domain, problem):
    actions = []
    # pour toute les actions
    for a in range(len(domain.actions)):
        action = copy.deepcopy(list(domain.actions)[a])
        possiblity=generate_klist(copy.deepcopy(list(problem.objects)), len(list(action.parameters)))
        # pour toutes les combinaisons de variables possible pour une action et si les 3 variables sont différentes
        for klist in possiblity :
            if len(set(klist))==len(klist):
                dict_action = {}
                # name
                dict_action['name'] = action.name.__str__()+'('+', '.join([v.__str__() for v in klist])+')'
                dict_var=dict(zip(action.parameters, klist))
                # preconditions
                dict_action['preconditions'] = []

                for p in range(len(list(list((domain.actions))[a].precondition.operands))):
                    precondition = copy.deepcopy(list(list((domain.actions))[a].precondition.operands)[:])[p]
                    terms =tuple([dict_var[v] for v in precondition.__getstate__()['_terms']])
                    precondition.__getstate__()['_terms']=terms
                    dict_action['preconditions'].append(precondition)

                effects=copy.deepcopy(list(list((domain.actions))[a].effect.operands))

                # positive effects
                dict_action['positive effects'] = []

                # negative effects
                dict_action['negative effects'] = []

                for e in range(len(effects)):
                    effect = effects[e]
                    if type(effect) is pddl.logic.predicates.Predicate:
                        terms = copy.deepcopy(tuple([dict_var[v] for v in effect.__getstate__()['_terms']]))
                        effect.__getstate__()['_terms']=terms
                        dict_action['positive effects'].append(effect)  

                    elif type(effect) is pddl.logic.base.Not:
                        terms =tuple([dict_var[v] for v in effect.__getstate__()['_arg'].__getstate__()['_terms']])
                        effect.__getstate__()['_arg'].__getstate__()['_terms']=terms
                        dict_action['negative effects'].append(effect)         
                            
                actions.append(dict_action) 
    return actions

# print(all_actions(domain, problem))

### Application d'actions sur state

In [87]:
def applicable_actions(state, actions): # return applicable actions for a state
    applicable_actions = []
    for a in actions:
        if all([p in state for p in a['preconditions']]):
            applicable_actions.append(a)
    return applicable_actions

print(applicable_actions(init_state, all_actions(domain, problem)))


def apply_action(state, action): # return new state after applying an action
    new_state = copy.deepcopy(state)
    for e in action['positive effects']:
        new_state.append(e)
    for e in action['negative effects']:
        new_state.remove(e.__getstate__()['_arg'])
    return new_state

print(apply_action(init_state, applicable_actions(init_state, all_actions(domain, problem))[0]))

def is_applicable(state, action): # return True if an action is applicable for a state
    copy_state = copy.deepcopy(state)   
    try:
        apply_action(copy_state, action)
        return True
    except:
        return False

# action=[d for d in all_actions(domain, problem) if d['name']=='move(d0, d1, p2)'][0]
# print(is_applicable(init_state, action))

def misapply_action(state, action): # return new state after misapplying an action
    new_state = copy.deepcopy(state)
    for e in action['positive effects']:
        new_state.remove(e)
    for e in action['negative effects']:
        new_state.append(e.__getstate__()['_arg'])
    return new_state


def previous_actions(state, actions): # return previous actions for a state
    previous_actions = []
    for a in actions:
        cdt1 = all([p in state for p in a['positive effects']])
        cdt2 = all([p.__getstate__()['_arg'] not in state for p in a['negative effects']])
        cdt3 = all([p in state for p in (set(a['preconditions'])-set([p.__getstate__()['_arg'] for p in a['negative effects']]))])
        if cdt1 and cdt2 and cdt3:
            try:
                misapply_action(state, a)
                previous_actions.append(a)
            except:
                pass
    return previous_actions

print(previous_actions(goal_state, all_actions(domain, problem)))

# print(misapply_action(problem.goal.__getstate__()['_operands'], previous_actions(problem.goal.__getstate__()['_operands'], all_actions(domain, problem))[0]))

[{'name': 'move(d0, d1, p1)', 'preconditions': [Predicate(smaller, d0, p1), Predicate(on, d0, d1), Predicate(clear, d0), Predicate(clear, p1)], 'positive effects': [Predicate(clear, d1), Predicate(on, d0, p1)], 'negative effects': [Not(Predicate(on, d0, d1)), Not(Predicate(clear, p1))]}, {'name': 'move(d0, d1, p2)', 'preconditions': [Predicate(smaller, d0, p2), Predicate(on, d0, d1), Predicate(clear, d0), Predicate(clear, p2)], 'positive effects': [Predicate(clear, d1), Predicate(on, d0, p2)], 'negative effects': [Not(Predicate(on, d0, d1)), Not(Predicate(clear, p2))]}]
[Predicate(smaller, d2, p2), Predicate(smaller, d2, p1), Predicate(smaller, d0, p0), Predicate(on, d2, d3), Predicate(smaller, d1, p0), Predicate(clear, p2), Predicate(smaller, d1, d3), Predicate(smaller, d0, d3), Predicate(smaller, d3, p0), Predicate(smaller, d1, p1), Predicate(smaller, d0, p1), Predicate(smaller, d3, p1), Predicate(on, d1, d2), Predicate(smaller, d0, p2), Predicate(smaller, d1, p2), Predicate(smaller,

### Heuristique

In [82]:
## Métrique
Initial = init_state.copy()
Goal =  goal_state.copy()

def metric(state, goal_state):
    return len(set(state)-set(goal_state))

metric(Initial, Goal)

2

In [81]:
def metric_relaxed(current_state, goal_state):
    # Prédicats présents dans l'état but mais absents de l'état actuel
    to_add = set(goal_state) - set(current_state)
    
    # La métrique est le nombre de prédicats à ajouter, ignorons les prédicats à retirer
    return len(to_add)

metric_relaxed(Initial, Goal)

2

In [80]:
def metric_improved(current_state, goal_state):
    # Prédicats présents dans l'état actuel mais absents de l'état but
    to_remove = set(current_state) - set(goal_state)
    
    # Prédicats présents dans l'état but mais absents de l'état actuel
    to_add = set(goal_state) - set(current_state)
    
    # La métrique est la somme du nombre de prédicats à ajouter et à retirer
    return len(to_remove) + len(to_add)

metric_improved(Initial, Goal)

4

In [94]:
# from collections import deque

def relaxed_plan_cost(initial_state, goal_state, domain, problem):
    all_possible_actions = all_actions(domain, problem)
    frontier = deque([initial_state])
    visited = set()
    state_to_cost = {tuple(initial_state): 0}  # Dictionary to track the cost to reach a state

    while frontier:
        current_state = frontier.popleft()
        current_state_tuple = tuple(current_state)
        
        if current_state_tuple in visited:
            continue
        visited.add(current_state_tuple)

        # Check if current state meets the goal state criteria
        if set(goal_state).issubset(set(current_state)):
            return state_to_cost[current_state_tuple]
        
        for action in applicable_actions(current_state, all_possible_actions):
            new_state = apply_action(current_state, action)
            new_state_tuple = tuple(new_state)
            
            # Ignore negative effects by not considering them in the new state generation
            if new_state_tuple not in state_to_cost or state_to_cost[new_state_tuple] > state_to_cost[current_state_tuple] + 1:
                state_to_cost[new_state_tuple] = state_to_cost[current_state_tuple] + 1
                frontier.append(new_state)

    return float('inf')  # Return infinity if no plan can be found

# Example usage:

def find_action_by_name(actions, name):
    for action in actions:
        if action['name'] == name:
            return action
    return None

# Exemple d'utilisation
actions_dict = all_actions(domain, problem)  # Votre liste de dictionnaires ici
action_name = 'move(d1, d2, p2)'  # Le nom de l'action que vous recherchez
action_found = find_action_by_name(actions_dict, action_name)


dict = all_actions(domain, problem)
state1= apply_action(init_state, applicable_actions(init_state, all_actions(domain, problem))[0])
state2= apply_action(state1, applicable_actions(state1, action_found))

cost = relaxed_plan_cost(state2, goal_state, domain, problem)
print(cost)


TypeError: 'list' object is not callable

### Fonction de vérification du planner

### Algo de planification LAMA

# Librairie unified-planning

### Solveur


In [5]:
! pip install unified-planning[fast-downward]

Defaulting to user installation because normal site-packages is not writeable


In [6]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *
from unified_planning.engines import PlanGenerationResultStatus

reader = PDDLReader()
pddl_problem = reader.parse_problem('data/domain.pddl', 'data/problem.pddl')
# print(pddl_problem)


with OneshotPlanner(
    problem_kind=pddl_problem.kind,
    optimality_guarantee=PlanGenerationResultStatus.SOLVED_OPTIMALLY,
) as planner:
    final_report = planner.solve(pddl_problem)
    plan = final_report.plan


print("Plan found")   
print(plan)

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 550 of `C:\Users\abdel\AppData\Roaming\Python\Python311\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

Plan found
SequentialPlan:
    move(d0, d1, p1)
    move(d1, d2, p2)
    move(d0, p1, d1)
    move(d2, d3, p1)
    move(d0, d1, d3)
    move(d1, p2, d2)
    move(d0, d3, d1)
    move(d3, p0, p2)
    move(d0, d1, d3)
    move(d1, d2, p0)
    move(d0, d3, d1)
    move(d2, p1, d3)
    move(d0, d1, p1)
    move(d1, p0, d2)
    move(d0, p1, d1)


### Actions applicables, paramètres applicables modification de l'état

In [7]:
# print(pddl_problem)
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *
from unified_planning.plans import ActionInstance

reader = PDDLReader()
pddl_problem = reader.parse_problem('data/domain.pddl', 'data/problem.pddl')

simulator = SequentialSimulator(pddl_problem)
init_state2=simulator.get_initial_state()
goal=pddl_problem.goals

(action_name, para) = tuple(simulator.get_applicable_actions(init_state2))[0] # récup de la première fonction applicable et des paramètre qui conviennent

action_instance = ActionInstance(action_name, para) #création de l'objet pour appliquer la fonction

new_state = simulator.apply(init_state2, action_instance) #on applique la fonction sur l'état

print(new_state)


{clear(d1): true, on(d0, p1): true, on(d0, d1): false, clear(p1): false, clear(d0): true, clear(p2): true, on(d1, d2): true, on(d2, d3): true, on(d3, p0): true, smaller(d0, d1): true, smaller(d0, d2): true, smaller(d0, d3): true, smaller(d0, p0): true, smaller(d0, p1): true, smaller(d0, p2): true, smaller(d1, d2): true, smaller(d1, d3): true, smaller(d1, p0): true, smaller(d1, p1): true, smaller(d1, p2): true, smaller(d2, d3): true, smaller(d2, p0): true, smaller(d2, p1): true, smaller(d2, p2): true, smaller(d3, p0): true, smaller(d3, p1): true, smaller(d3, p2): true, clear(d2): false, clear(d3): false, clear(p0): false, on(d0, d0): false, on(d1, d0): false, on(d2, d0): false, on(d3, d0): false, on(p0, d0): false, on(p1, d0): false, on(p2, d0): false, on(d1, d1): false, on(d2, d1): false, on(d3, d1): false, on(p0, d1): false, on(p1, d1): false, on(p2, d1): false, on(d0, d2): false, on(d2, d2): false, on(d3, d2): false, on(p0, d2): false, on(p1, d2): false, on(p2, d2): false, on(d0, d3)

# Librairie pddlpy

In [8]:
import pddlpy

# Load your domain and problem PDDL files
domprob = pddlpy.DomainProblem('data/domain.pddl', 'data/problem.pddl')
print(domprob.worldobjects())
print(domprob.initialstate())
print(domprob.goals())
print(domprob.operators())
print(domprob.ground_operator('move'))
print(domprob.domain.operators['move'])

help(domprob.ground_operator('move'))

{'p0': None, 'd2': None, 'p1': None, 'd1': None, 'd3': None, 'p2': None, 'smaller': None, 'on': None, 'd0': None, 'clear': None}
{('smaller', 'd1', 'p1'), ('smaller', 'd0', 'p0'), ('on', 'd1', 'd2'), ('smaller', 'd3', 'p1'), ('smaller', 'd1', 'p2'), ('smaller', 'd0', 'p1'), ('on', 'd2', 'd3'), ('smaller', 'd3', 'p2'), ('smaller', 'd2', 'd3'), ('smaller', 'd0', 'p2'), ('on', 'd3', 'p0'), ('smaller', 'd2', 'p0'), ('smaller', 'd3', 'p0'), ('smaller', 'd1', 'd2'), ('clear', 'd0'), ('smaller', 'd0', 'd1'), ('smaller', 'd2', 'p1'), ('clear', 'p1'), ('smaller', 'd1', 'd3'), ('smaller', 'd0', 'd2'), ('clear', 'p2'), ('smaller', 'd2', 'p2'), ('smaller', 'd1', 'p0'), ('smaller', 'd0', 'd3'), ('on', 'd0', 'd1')}
{('smaller', 'd0', 'p0'), ('smaller', 'd3', 'p1'), ('smaller', 'd1', 'p2'), ('clear', 'p1'), ('smaller', 'd0', 'p1'), ('smaller', 'd3', 'p2'), ('clear', 'd0'), ('smaller', 'd2', 'd3'), ('smaller', 'd0', 'p2'), ('on', 'd0', 'd1'), ('smaller', 'd0', 'd1'), ('smaller', 'd2', 'p0'), ('smaller